how to measure competition?

        pure open: 

        pure closed: 

        open plus closed: 
    

In [17]:
# preparation ...
import pandas as pd
from tqdm import tqdm
import itertools
from collections import Counter

# we can not find 'PanGU-Σ', 'Luminous' in close Source index, cannot find 'Galatica', 'YaLM', 'PanGu-α' in open source list , so delete them.
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Llama','Vicuna','ChatGLM','CodeGeeX','Koala','GPT-2','GPT2','GPT 2']
closeSourceList = ['GShard','GPT-3','GPT 3', 'GPT3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4', 'GPT4','Claude']

readfile = "/data/jx4237data/DataForChatGPTinnovationWaves/"
df = pd.read_csv(readfile + 'LLM09enhanced.csv',dtype=object)
# some paper we need to delete:
# “fake” palm paper 1909.02134
# fake openai 1506.04006
delete_list = ['1506.04006','1909.02134']
df = df[~df['id'].isin(delete_list)]
# model2paper
model2paper = {}
for model in tqdm(openSourceList+closeSourceList):
    if model in model2paper:
        print('error')
    model2paper[model] = set()
    for paper in df[df['abstract'].str.contains("\\b%s\\b"%model) | df['abstract'].str.lower().str.contains("\\b%s\\b"%model.lower())  | df['title'].str.contains("\\b%s\\b"%model) | df['title'].str.lower().str.contains("\\b%s\\b"%model.lower())]['id']  :
        model2paper[model].add(paper)

for i in model2paper['GPT 4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT 3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT2']:
    model2paper['GPT-2'].add(i)
for i in model2paper['GPT 2']:
    model2paper['GPT-2'].add(i)
for i in model2paper['Llama']:
    model2paper['LLaMA'].add(i)
del model2paper['GPT 4']
del model2paper['GPT4']
del model2paper['GPT3']
del model2paper['GPT 3']
del model2paper['GPT2']
del model2paper['GPT 2']
del model2paper['Llama']

# paper2model
paper2model = {}
for model, paperSet in model2paper.items():
    for paper in paperSet:
        if paper not in paper2model:
            paper2model[paper] =[model]
        else:
            paper2model[paper].append(model)

openPaperSet = set()
closePaperSet = set()
for k,v in model2paper.items():
    if k in closeSourceList:
        for paper in v:
            closePaperSet.add(paper)
    if k in openSourceList:
        for paper in v:
            openPaperSet.add(paper)
pureOpen = openPaperSet - closePaperSet
pureClose = closePaperSet - openPaperSet
mixed = openPaperSet & closePaperSet
all = openPaperSet | closePaperSet

paper2pubdate = {}
for i in range(len(df)):
    paper2pubdate[list(df['id'])[i]] = list(df['update_date'])[i]

100%|██████████| 60/60 [00:21<00:00,  2.82it/s]


In [18]:
# number of papers
from tabulate import tabulate
temp = [{'item':'total papers','freq':len(df)},{'item':'papers containing LLM','freq':3228},{'item':'only open models','freq':len(pureOpen)},{'item':'only closed models','freq':len(pureClose)},{'item':'mixed','freq':len(mixed)},]
print(tabulate(temp, headers="keys", tablefmt="fancy_grid"))

╒═══════════════════════╤════════╕
│ item                  │   freq │
╞═══════════════════════╪════════╡
│ total papers          │   6676 │
├───────────────────────┼────────┤
│ papers containing LLM │   3228 │
├───────────────────────┼────────┤
│ only open models      │    881 │
├───────────────────────┼────────┤
│ only closed models    │   2264 │
├───────────────────────┼────────┤
│ mixed                 │    409 │
╘═══════════════════════╧════════╛


### Descriptive Analysis

propotion of each mentioned model

In [19]:

model_name_list = list(model2paper.keys())

# 总共共现出现的次数
ackTotalCounter = Counter()
for k,v in paper2model.items():
    if len(v) > 1:
        for model in v:
            ackTotalCounter[model] += (len(v)-1)

# open出现的数量
ackOpenCounter = Counter()
for k,v in paper2model.items():
    if len(v) > 1:
        for model in v:
            for other_model in v:
                if other_model == model:
                    continue
                elif other_model in openSourceList:
                    ackOpenCounter[model] += 1

singleNameCounter = Counter()
for k,v in paper2model.items():
    if len(v) == 1:
        for model in v:
            singleNameCounter[model] += 1

doubleNameCounter = Counter()
for k,v in paper2model.items():
    if len(v) == 2:
        for model in v:
            doubleNameCounter[model] += 1

tripleNameCounter = Counter()
for k,v in paper2model.items():
    if len(v) == 3:
        for model in v:
            tripleNameCounter[model] += 1

quadrupleNameCounter = Counter()
for k,v in paper2model.items():
    if len(v) == 4:
        for model in v:
            quadrupleNameCounter[model] += 1

print('All models:')
rank = 1
modelMentionTimes4tableList = []
for k,v in sorted(model2paper.items(), key= lambda x:-len(x[1])):
    modelMentionTimes4tableList.append({'Rank':rank,'Model':k,'Mentioned':len(v), 'Type':'open' if k in openSourceList else 'closed','% of open-ack':round(ackOpenCounter[k]/ackTotalCounter[k],3) if ackTotalCounter[k]!=0 else 'NA' ,'single':singleNameCounter[k],'double':doubleNameCounter[k],'triple':tripleNameCounter[k], 'quadruple':quadrupleNameCounter[k],'% of single mention':round(singleNameCounter[k]/len(v), 3),'% of 3228':round(len(v)/3228, 3)})
    rank += 1
print(tabulate(modelMentionTimes4tableList, headers="keys", tablefmt="fancy_grid"))

print('Open models:')
rank = 1
modelMentionTimes4tableList = []
for k,v in sorted(model2paper.items(), key= lambda x:-len(x[1])):
    if k in closeSourceList:
        continue
    modelMentionTimes4tableList.append({'Rank':rank,'Model':k,'Mentioned':len(v), 'Type':'open' if k in openSourceList else 'closed','% of open-ack':round(ackOpenCounter[k]/ackTotalCounter[k],3) if ackTotalCounter[k]!=0 else 'NA' ,'single':singleNameCounter[k],'double':doubleNameCounter[k],'triple':tripleNameCounter[k], 'quadruple':quadrupleNameCounter[k],'% of single mention':round(singleNameCounter[k]/len(v), 3),'% of 3228':round(len(v)/3228, 3)})
    rank += 1
print(tabulate(modelMentionTimes4tableList, headers="keys", tablefmt="fancy_grid"))

print('Closed models:')
rank = 1
modelMentionTimes4tableList = []
for k,v in sorted(model2paper.items(), key= lambda x:-len(x[1])):
    if k in openSourceList:
        continue
    modelMentionTimes4tableList.append({'Rank':rank,'Model':k,'Mentioned':len(v), 'Type':'open' if k in openSourceList else 'closed','% of open-ack':round(ackOpenCounter[k]/ackTotalCounter[k],3) if ackTotalCounter[k]!=0 else 'NA' ,'single':singleNameCounter[k],'double':doubleNameCounter[k],'triple':tripleNameCounter[k], 'quadruple':quadrupleNameCounter[k],'% of single mention':round(singleNameCounter[k]/len(v), 3),'% of 3228':round(len(v)/3228, 3)})
    rank += 1
print(tabulate(modelMentionTimes4tableList, headers="keys", tablefmt="fancy_grid"))


All models:
╒════════╤═════════════════╤═════════════╤════════╤═════════════════╤══════════╤══════════╤══════════╤═════════════╤═══════════════════════╤═════════════╕
│   Rank │ Model           │   Mentioned │ Type   │ % of open-ack   │   single │   double │   triple │   quadruple │   % of single mention │   % of 3228 │
╞════════╪═════════════════╪═════════════╪════════╪═════════════════╪══════════╪══════════╪══════════╪═════════════╪═══════════════════════╪═════════════╡
│      1 │ ChatGPT         │        1264 │ closed │ 0.262           │      854 │      271 │       98 │          24 │                 0.676 │       0.392 │
├────────┼─────────────────┼─────────────┼────────┼─────────────────┼──────────┼──────────┼──────────┼─────────────┼───────────────────────┼─────────────┤
│      2 │ GPT-3           │        1027 │ closed │ 0.336           │      542 │      322 │      107 │          35 │                 0.528 │       0.318 │
├────────┼─────────────────┼─────────────┼────────┼───────

distribution of # models

In [20]:

modelDistribution = Counter()
for k,v in paper2model.items():
    modelDistribution[len(v)] += 1
modelDistributionlist = []
for k,v in sorted(modelDistribution.items()):
    modelDistributionlist.append({'Number of models a paper has':k,'freq':v})

print(tabulate(modelDistributionlist, headers="keys", tablefmt="fancy_grid"))
sum(modelDistribution.values())

╒════════════════════════════════╤════════╕
│   Number of models a paper has │   freq │
╞════════════════════════════════╪════════╡
│                              1 │   2586 │
├────────────────────────────────┼────────┤
│                              2 │    665 │
├────────────────────────────────┼────────┤
│                              3 │    190 │
├────────────────────────────────┼────────┤
│                              4 │     70 │
├────────────────────────────────┼────────┤
│                              5 │     24 │
├────────────────────────────────┼────────┤
│                              6 │     13 │
├────────────────────────────────┼────────┤
│                              7 │      4 │
├────────────────────────────────┼────────┤
│                              8 │      1 │
├────────────────────────────────┼────────┤
│                              9 │      1 │
╘════════════════════════════════╧════════╛


3554

combination

In [21]:
# overall
def gen_pair(input_list):
    output_list = []
    for each_pair in list(itertools.combinations(sorted(input_list), 2)):

        output_list.append(' - '.join(each_pair))

    return output_list

double_counter = Counter()

for k,v in paper2model.items():
    if len(v) > 1:
        for combination in gen_pair(v):
            double_counter[combination] += 1

CombineList = []
for k,v in sorted(double_counter.items(), key=lambda x  : -x[1]):
    CombineList.append({'Model Combination':k,'freq':v})
print('')
print(tabulate(CombineList, headers="keys", tablefmt="fancy_grid"))


╒═══════════════════════════╤════════╕
│ Model Combination         │   freq │
╞═══════════════════════════╪════════╡
│ ChatGPT - GPT-4           │    212 │
├───────────────────────────┼────────┤
│ GPT-3 - GPT-4             │    176 │
├───────────────────────────┼────────┤
│ ChatGPT - GPT-3           │    155 │
├───────────────────────────┼────────┤
│ GPT-3 - T5                │     69 │
├───────────────────────────┼────────┤
│ FLAN - T5                 │     61 │
├───────────────────────────┼────────┤
│ GPT-4 - LLaMA             │     59 │
├───────────────────────────┼────────┤
│ ChatGPT - LLaMA           │     58 │
├───────────────────────────┼────────┤
│ FLAN - Flan-T5            │     57 │
├───────────────────────────┼────────┤
│ Flan-T5 - T5              │     57 │
├───────────────────────────┼────────┤
│ GPT-3 - LLaMA             │     48 │
├───────────────────────────┼────────┤
│ GPT-2 - GPT-3             │     46 │
├───────────────────────────┼────────┤
│ Codex - GPT-3         

t5, flan-t5, and mt5


t5 (https://huggingface.co/docs/transformers/v4.32.1/en/model_doc/t5#overview
): 2020-6, T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, e.g., for translation: translate English to German: …, for summarization: summarize: ….

T5 comes in different sizes:

    t5-small
    t5-base
    t5-large
    t5-3b
    t5-11b

Based on the original T5 model, Google has released some follow-up works:


    mT5: mT5 is a multilingual T5 model. It is pre-trained on the mC4 corpus, which includes 101 languages

    UL2: UL2 is a T5 like model pretrained on various denoising objectives

    Flan-T5: Flan is a pretraining methods that is based on prompting. The Flan-T5 are T5 models trained on the Flan collection of datasets which include: taskmaster2, djaym7/wiki_dialog, deepmind/code_contests, lambada, gsm8k, aqua_rat, esnli, quasc and qed.

    FLan-UL2 : the UL2 model finetuned using the “Flan” prompt tuning and dataset collection.

    UMT5: UmT5 is a multilingual T5 model trained on an improved and refreshed mC4 multilingual corpus, 29 trillion characters across 107 language, using a new sampling method, UniMax.

### open model

In [22]:
# open  models
double_counter = Counter()
pd.set_option('display.max_colwidth', None)


for k,v in paper2model.items():
    if k in pureOpen and len(v) > 1:
        for combination in gen_pair(v):
            double_counter[combination] += 1
        # if 'GPT-2' in v and 'OPT' in v:
        #     print(df[df['id'] == k][['publish_date_v1','title','abstract']].iloc[0].values)

CombineList = []
for k,v in sorted(double_counter.items(), key=lambda x  : -x[1]):
    CombineList.append({'Open Model Combination':k,'freq':v})
print('')
print(tabulate(CombineList, headers="keys", tablefmt="fancy_grid"))


╒══════════════════════════╤════════╕
│ Open Model Combination   │   freq │
╞══════════════════════════╪════════╡
│ GPT-2 - T5               │     20 │
├──────────────────────────┼────────┤
│ LLaMA - OPT              │      9 │
├──────────────────────────┼────────┤
│ GPT-2 - OPT              │      9 │
├──────────────────────────┼────────┤
│ BLOOM - OPT              │      6 │
├──────────────────────────┼────────┤
│ T5 - mT5                 │      5 │
├──────────────────────────┼────────┤
│ BLOOM - LLaMA            │      5 │
├──────────────────────────┼────────┤
│ GPT-2 - LLaMA            │      5 │
├──────────────────────────┼────────┤
│ BLOOM - BLOOMZ           │      4 │
├──────────────────────────┼────────┤
│ LLaMA - Vicuna           │      4 │
├──────────────────────────┼────────┤
│ LLaMA - T5               │      3 │
├──────────────────────────┼────────┤
│ OPT - T0                 │      2 │
├──────────────────────────┼────────┤
│ OPT - Vicuna             │      2 │
├──────────

bloom-t5:

both are large open source models: OPT-175B and BLOOM-176B

With the release of BLOOM-176B and OPT-175B, everyone can download pretrained models of this scale.




**bloom (https://huggingface.co/bigscience/bloom)**

BigScience Large Open-science Open-access Multilingual Language Model (paper: 9 Nov 2022; model: july, 2021)

Current Checkpoint: Training Iteration 95000

Total seen tokens: 366B

BLOOM is an autoregressive Large Language Model (LLM), trained to continue text from a prompt on vast amounts of text data using industrial-scale computational resources. As such, it is able to output coherent text in 46 languages and 13 programming languages that is hardly distinguishable from text written by humans. BLOOM can also be instructed to perform text tasks it hasn't been explicitly trained for, by casting them as text generation tasks.

Funded by:

    The French government.

    Hugging Face (website).

**OPT (https://huggingface.co/facebook/opt-2.7b)**

OPT : Open Pre-trained Transformer Language Models
OPT was first introduced in Open Pre-trained Transformer Language Models and first released in metaseq's repository on May 3rd 2022 by **Meta AI**.

### Closed model

In [23]:
# open  models
double_counter = Counter()
pd.set_option('display.max_colwidth', None)


for k,v in paper2model.items():
    if k in pureClose and len(v) > 1:
        for combination in gen_pair(v):
            double_counter[combination] += 1
        # if 'LLaMA' in v and 'OPT' in v:
        #     print(df[df['id'] == k][['publish_date_v1','title','abstract']].iloc[0].values)

CombineList = []
for k,v in sorted(double_counter.items(), key=lambda x  : -x[1]):
    CombineList.append({'Closed Model Combination':k,'freq':v})
print('')
print(tabulate(CombineList, headers="keys", tablefmt="fancy_grid"))


╒════════════════════════════╤════════╕
│ Closed Model Combination   │   freq │
╞════════════════════════════╪════════╡
│ ChatGPT - GPT-4            │    177 │
├────────────────────────────┼────────┤
│ GPT-3 - GPT-4              │    136 │
├────────────────────────────┼────────┤
│ ChatGPT - GPT-3            │    132 │
├────────────────────────────┼────────┤
│ Codex - GPT-3              │     32 │
├────────────────────────────┼────────┤
│ GPT-3 - PaLM               │     17 │
├────────────────────────────┼────────┤
│ GPT-3 - InstructGPT        │     14 │
├────────────────────────────┼────────┤
│ ChatGPT - Codex            │     13 │
├────────────────────────────┼────────┤
│ ChatGPT - InstructGPT      │     12 │
├────────────────────────────┼────────┤
│ GPT-4 - PaLM               │     10 │
├────────────────────────────┼────────┤
│ ChatGPT - Claude           │     10 │
├────────────────────────────┼────────┤
│ Claude - GPT-4             │      9 │
├────────────────────────────┼────────┤

### mixed model

In [24]:
# open  models
double_counter = Counter()
pd.set_option('display.max_colwidth', None)


for k,v in paper2model.items():
    if k in mixed and len(v) > 1:
        for combination in gen_pair(v):
            double_counter[combination] += 1
        # if 'LLaMA' in v and 'OPT' in v:
        #     print(df[df['id'] == k][['publish_date_v1','title','abstract']].iloc[0].values)

CombineList = []
for k,v in sorted(double_counter.items(), key=lambda x  : -x[1]):
    CombineList.append({'Closed Model Combination':k,'freq':v})
print('')
print(tabulate(CombineList, headers="keys", tablefmt="fancy_grid"))


╒════════════════════════════╤════════╕
│ Closed Model Combination   │   freq │
╞════════════════════════════╪════════╡
│ GPT-3 - T5                 │     69 │
├────────────────────────────┼────────┤
│ FLAN - T5                  │     61 │
├────────────────────────────┼────────┤
│ GPT-4 - LLaMA              │     59 │
├────────────────────────────┼────────┤
│ ChatGPT - LLaMA            │     58 │
├────────────────────────────┼────────┤
│ FLAN - Flan-T5             │     57 │
├────────────────────────────┼────────┤
│ Flan-T5 - T5               │     57 │
├────────────────────────────┼────────┤
│ GPT-3 - LLaMA              │     48 │
├────────────────────────────┼────────┤
│ GPT-2 - GPT-3              │     46 │
├────────────────────────────┼────────┤
│ GPT-3 - GPT-4              │     40 │
├────────────────────────────┼────────┤
│ ChatGPT - GPT-4            │     35 │
├────────────────────────────┼────────┤
│ Alpaca - LLaMA             │     26 │
├────────────────────────────┼────────┤